In [18]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
import pandas as pd
import numpy as np
from sklearnex import patch_sklearn
from daal4py.oneapi import sycl_context #for speed up GPU
patch_sklearn()
sycl_context("gpu")
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [19]:
data=pd.read_csv("train.csv")
data.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [20]:
x=data.iloc[:,:20].values
y=data.iloc[:,20:21].values
#print(y)

In [21]:
sc= StandardScaler()
x=sc.fit_transform(x)

In [22]:
print(x)

[[-0.90259726 -0.9900495   0.83077942 ... -1.78686097 -1.00601811
   0.98609664]
 [-0.49513857  1.0100505  -1.2530642  ...  0.55964063  0.99401789
  -1.01409939]
 [-1.5376865   1.0100505  -1.2530642  ...  0.55964063  0.99401789
  -1.01409939]
 ...
 [ 1.53077336 -0.9900495  -0.76274805 ...  0.55964063  0.99401789
  -1.01409939]
 [ 0.62252745 -0.9900495  -0.76274805 ...  0.55964063  0.99401789
   0.98609664]
 [-1.65833069  1.0100505   0.58562134 ...  0.55964063  0.99401789
   0.98609664]]


In [23]:
ohe=OneHotEncoder()
y=ohe.fit_transform(y).toarray()

In [24]:
y

array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.]])

In [25]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.1)

In [26]:
#nn
nn=Sequential()
nn.add(Dense(16, input_dim=20, activation="relu"))
nn.add(Dense(12, activation="relu"))
nn.add(Dense(4,activation="softmax"))

In [27]:
nn.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [28]:
tnn=nn.fit(x_train,y_train,epochs=100,batch_size=64)

Epoch 1/100
29/29 [==============================] - 1s 1ms/step - loss: 1.4152 - accuracy: 0.2694
Epoch 2/100
29/29 [==============================] - 0s 1ms/step - loss: 1.3420 - accuracy: 0.3117
Epoch 3/100
29/29 [==============================] - 0s 1ms/step - loss: 1.2838 - accuracy: 0.3700
Epoch 4/100
29/29 [==============================] - 0s 1ms/step - loss: 1.2201 - accuracy: 0.4244
Epoch 5/100
29/29 [==============================] - 0s 1ms/step - loss: 1.1411 - accuracy: 0.4967
Epoch 6/100
29/29 [==============================] - 0s 1ms/step - loss: 1.0495 - accuracy: 0.5583
Epoch 7/100
29/29 [==============================] - 0s 1ms/step - loss: 0.9531 - accuracy: 0.6100
Epoch 8/100
29/29 [==============================] - 0s 1ms/step - loss: 0.8605 - accuracy: 0.6478
Epoch 9/100
29/29 [==============================] - 0s 1ms/step - loss: 0.7783 - accuracy: 0.7011
Epoch 10/100
29/29 [==============================] - 0s 1ms/step - loss: 0.7073 - accuracy: 0.7306
Epoch 11/

In [29]:
y_pred=nn.predict(x_test)

In [30]:
pred=list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))
test=list()
for i in range(len(y_test)):
    test.append(np.argmax(y_test[i]))

In [31]:
accuracy_score(pred,test)

0.935